### Assignment : Week 1 
## Modeling simple RL problems by making their MDPs in Python

We will create the MDPs for some of the example problems from Grokking textbook. For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

Later on, you will not need to implement the MDPs of common RL problems yourself, most of the work is already done by the OpenAI Gym library, which includes models for most of the famous RL envis.

You can start this assignment during/after reading Grokking Ch-2.

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [1]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Let us now look at the BSW environment on Page 40. We'll model a slightly modified version of BSW with 7 states instead (i.e the SWF envi on Page 67). It will be useful in the coming weeks.

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [2]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    6 : {
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    },
    
}

# I didn't exactly get what was meant by "two different tuples have positive rewards, you need to find them."

for i in range(5):
    leftmost = (i == 0)
    rightmost = (i == 4)

    swf_mdp[i + 1] = {
        "Right": [
            (1/2, i+2, 1 * rightmost, rightmost),
            (1/3, i+1, 0, False),
            (1/6, i, 0, leftmost)
        ],
        "Left": [
            (1/2, i, 0, leftmost),
            (1/3, i+1, 0, False),
            (1/6, i, 1 * rightmost, rightmost)
        ]
    }

Feel free to automate filling this MDP, but ensure that it is correctly filled as it'll be back in next week's assignment.

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [3]:
# special states
def is_terminal(state):
    return state in [5, 7, 11, 12, 15]

def is_reward(state):
    return state in [15]

# border states
def is_rightmost(state):
    return state % 4 == 3

def is_leftmost(state):
    return state % 4 == 0

def is_topmost(state):
    return state < 4

def is_bottommost(state):
    return state > 11


def generate_move_tuple(state: int, direction: str):
    if is_terminal(state):
        return (
            1.,
            state,
            0,
            True
        )
    
    if direction == "Up":
        new_state = state - 4 * (not is_topmost(state))
    elif direction == "Down":
        new_state = state + 4 * (not is_bottommost(state))
    elif direction == "Right":
        new_state = state + 1 * (not is_rightmost(state))   
    elif direction == "Left":
        new_state = state - 1 * (not is_leftmost(state))
    else:
        raise ValueError("Error: Invalid Direction")

    return (
            1/3,
            new_state,
            1 * (not is_reward(state) and is_reward(new_state)),
            is_terminal(new_state)
        )

def generate_move_list(state: int, chosen_direction: str):
    if is_terminal(state):
        return [generate_move_tuple(state, "")]
    
    if chosen_direction == "Up" or chosen_direction == "Down":
        ortho_direction1 = "Right"
        ortho_direction2 = "Left"
    elif chosen_direction == "Right" or chosen_direction == "Left":
        ortho_direction1 = "Up"
        ortho_direction2 = "Down"
    else:
        raise ValueError(f"Error: Invalid Direction '{chosen_direction}'")

    return [
        generate_move_tuple(state, chosen_direction),
        generate_move_tuple(state, ortho_direction1),
        generate_move_tuple(state, ortho_direction2),
    ]


In [4]:
fl_mdp = {
    # to be added
}

In [5]:
for state in range(0, 16):

    # add transitions to other states
    transitions = {}

    for action in ["Up", "Down", "Right", "Left"]:
        transitions[action] = generate_move_list(state, action)

    fl_mdp[state] = transitions

In [6]:
fl_mdp

{0: {'Up': [(0.3333333333333333, 0, 0, False),
   (0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 0, 0, False)],
  'Down': [(0.3333333333333333, 4, 0, False),
   (0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 0, 0, False)],
  'Right': [(0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 0, 0, False),
   (0.3333333333333333, 4, 0, False)],
  'Left': [(0.3333333333333333, 0, 0, False),
   (0.3333333333333333, 0, 0, False),
   (0.3333333333333333, 4, 0, False)]},
 1: {'Up': [(0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 2, 0, False),
   (0.3333333333333333, 0, 0, False)],
  'Down': [(0.3333333333333333, 5, 0, True),
   (0.3333333333333333, 2, 0, False),
   (0.3333333333333333, 0, 0, False)],
  'Right': [(0.3333333333333333, 2, 0, False),
   (0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 5, 0, True)],
  'Left': [(0.3333333333333333, 0, 0, False),
   (0.3333333333333333, 1, 0, False),
   (0.3333333333333333, 5, 0, True)]},
 2: {'Up': [(

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.

You don't need to worry about Gym right now, we'll set it up in the coming weeks. But here is the code to import an MDP.

In [ ]:
import gym
P = gym.make('FrozenLake-v1').env.P

Since the imported MDP is also just a dictionary, we can just print it.

In [ ]:
# using the pretty print module

import pprint
pprint.pprint(P)